# Day 3 - Afternoon Session: Final Integration Project
## Discovering a Resonance in Dimuon Events

**Duration:** 3.5 hours  
**Format:** Team project (pairs)

---

## Project Overview

You have simulated data from a particle detector recording muon pairs from proton-proton collisions. Your goal is to **identify a resonance peak** (like J/ψ or Z boson) in the invariant mass spectrum.

This project brings together everything learned in the course:
- Data loading and exploration (Day 1)
- NumPy/Pandas operations (Days 1-2)
- Visualization (Day 2)
- Functions and classes (Day 2)
- Error handling and testing (Day 3)

---

## Setup: Generate Sample Data

First, let's generate realistic dimuon event data for the analysis.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)
%matplotlib inline
sns.set_theme(style='whitegrid')

In [ ]:
# Generate simulated dimuon data
def generate_dimuon_data(n_events=10000, signal_fraction=0.3, z_mass=91.2, z_width=2.5):
    """
    Generate simulated dimuon events.
    
    Signal: Z boson decays (Gaussian mass distribution)
    Background: Continuum (exponential + flat)
    """
    n_signal = int(n_events * signal_fraction)
    n_background = n_events - n_signal
    
    events = []
    
    # Generate signal events (Z boson)
    for i in range(n_signal):
        # Generate mass from Breit-Wigner (approximated by Gaussian)
        target_mass = np.random.normal(z_mass, z_width)
        
        # Generate realistic kinematics that give this mass
        pt1 = np.random.exponential(35) + 20
        eta1 = np.random.uniform(-2.4, 2.4)
        phi1 = np.random.uniform(-np.pi, np.pi)
        
        # Calculate mu2 kinematics to achieve target mass
        pt2 = np.random.exponential(35) + 20
        eta2 = np.random.uniform(-2.4, 2.4)
        # Adjust phi2 to get approximately correct mass
        deta = eta1 - eta2
        cos_dphi = (target_mass**2 / (2 * pt1 * pt2) - np.cosh(deta)) / (-1)
        cos_dphi = np.clip(cos_dphi, -1, 1)
        dphi = np.arccos(cos_dphi)
        phi2 = phi1 + dphi + np.random.normal(0, 0.1)
        
        events.append({
            'event_id': i,
            'mu1_pt': pt1,
            'mu1_eta': eta1,
            'mu1_phi': phi1,
            'mu2_pt': pt2,
            'mu2_eta': eta2,
            'mu2_phi': phi2,
            'weight': 1.0,
            'is_signal': True
        })
    
    # Generate background events
    for i in range(n_background):
        pt1 = np.random.exponential(25) + 5
        pt2 = np.random.exponential(25) + 5
        eta1 = np.random.uniform(-2.5, 2.5)
        eta2 = np.random.uniform(-2.5, 2.5)
        phi1 = np.random.uniform(-np.pi, np.pi)
        phi2 = np.random.uniform(-np.pi, np.pi)
        
        events.append({
            'event_id': n_signal + i,
            'mu1_pt': pt1,
            'mu1_eta': eta1,
            'mu1_phi': phi1,
            'mu2_pt': pt2,
            'mu2_eta': eta2,
            'mu2_phi': phi2,
            'weight': 1.0,
            'is_signal': False
        })
    
    df = pd.DataFrame(events)
    return df.sample(frac=1).reset_index(drop=True)  # Shuffle

# Generate data
data = generate_dimuon_data(n_events=10000)

# Generate Monte Carlo (for comparison)
mc_data = generate_dimuon_data(n_events=50000, signal_fraction=0.3)

print(f"Data events: {len(data)}")
print(f"MC events: {len(mc_data)}")
data.head()

---
## Part 1: Data Loading and Exploration (45 min)

### Beginner Level
1. Inspect the data structure
2. Check for missing values and basic statistics
3. Create simple distribution plots
4. Calculate invariant mass

In [ ]:
# TODO: Explore the data structure

# YOUR CODE HERE
# 1. Print data.info()
# 2. Print data.describe()
# 3. Check for missing values with data.isnull().sum()

<details>
<summary>💡 Click to reveal solution</summary>

```python
# Data structure
print("=== Data Info ===")
print(data.info())

print("\n=== Statistics ===")
print(data.describe())

print("\n=== Missing Values ===")
print(data.isnull().sum())

print("\n=== Signal/Background split ===")
print(data['is_signal'].value_counts())
```

</details>

In [ ]:
# TODO: Create distribution plots for pT and eta

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# YOUR CODE HERE
# Plot 1: mu1_pt distribution
# Plot 2: mu2_pt distribution
# Plot 3: mu1_eta distribution
# Plot 4: mu2_eta distribution

plt.tight_layout()
plt.show()

<details>
<summary>💡 Click to reveal solution</summary>

```python
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# pT distributions
axes[0, 0].hist(data['mu1_pt'], bins=50, range=(0, 150), histtype='step', linewidth=2)
axes[0, 0].set_xlabel(r'$p_T^{\mu_1}$ (GeV)')
axes[0, 0].set_ylabel('Events')
axes[0, 0].set_title('Muon 1 pT Distribution')

axes[0, 1].hist(data['mu2_pt'], bins=50, range=(0, 150), histtype='step', linewidth=2)
axes[0, 1].set_xlabel(r'$p_T^{\mu_2}$ (GeV)')
axes[0, 1].set_ylabel('Events')
axes[0, 1].set_title('Muon 2 pT Distribution')

# eta distributions
axes[1, 0].hist(data['mu1_eta'], bins=50, range=(-3, 3), histtype='step', linewidth=2)
axes[1, 0].set_xlabel(r'$\eta^{\mu_1}$')
axes[1, 0].set_ylabel('Events')
axes[1, 0].set_title('Muon 1 eta Distribution')

axes[1, 1].hist(data['mu2_eta'], bins=50, range=(-3, 3), histtype='step', linewidth=2)
axes[1, 1].set_xlabel(r'$\eta^{\mu_2}$')
axes[1, 1].set_ylabel('Events')
axes[1, 1].set_title('Muon 2 eta Distribution')

plt.tight_layout()
plt.show()
```

</details>

In [ ]:
# TODO: Calculate invariant mass for each event

def calculate_invariant_mass(row):
    """
    Calculate invariant mass of a muon pair.
    
    Formula: M² = 2 * pT1 * pT2 * (cosh(Δη) - cos(Δφ))
    """
    # YOUR CODE HERE
    pass

# Apply to data
# data['mass'] = data.apply(calculate_invariant_mass, axis=1)

# Print statistics
# print(f"Mass range: [{data['mass'].min():.1f}, {data['mass'].max():.1f}] GeV")
# print(f"Mean mass: {data['mass'].mean():.1f} GeV")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def calculate_invariant_mass(row):
    """
    Calculate invariant mass of a muon pair.
    """
    deta = row['mu1_eta'] - row['mu2_eta']
    dphi = row['mu1_phi'] - row['mu2_phi']
    
    m2 = 2 * row['mu1_pt'] * row['mu2_pt'] * (np.cosh(deta) - np.cos(dphi))
    return np.sqrt(m2) if m2 > 0 else 0

# Apply to data
data['mass'] = data.apply(calculate_invariant_mass, axis=1)
mc_data['mass'] = mc_data.apply(calculate_invariant_mass, axis=1)

# Print statistics
print(f"Mass range: [{data['mass'].min():.1f}, {data['mass'].max():.1f}] GeV")
print(f"Mean mass: {data['mass'].mean():.1f} GeV")
print(f"Median mass: {data['mass'].median():.1f} GeV")
```

</details>

In [ ]:
# TODO: Plot the raw invariant mass distribution

fig, ax = plt.subplots(figsize=(10, 7))

# YOUR CODE HERE
# Create histogram of mass
# Use bins=100, range=(0, 200)

ax.set_xlabel(r'$m_{\mu\mu}$ (GeV/c²)', fontsize=14)
ax.set_ylabel('Events', fontsize=14)
ax.set_title('Dimuon Invariant Mass (Before Cuts)', fontsize=16)

plt.show()

<details>
<summary>💡 Click to reveal solution</summary>

```python
fig, ax = plt.subplots(figsize=(10, 7))

ax.hist(data['mass'], bins=100, range=(0, 200), histtype='step', 
        linewidth=2, color='black', label='Data')

ax.axvline(91.2, color='red', linestyle='--', alpha=0.7, label='Z mass (91.2 GeV)')

ax.set_xlabel(r'$m_{\mu\mu}$ (GeV/c²)', fontsize=14)
ax.set_ylabel('Events', fontsize=14)
ax.set_title('Dimuon Invariant Mass (Before Cuts)', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.show()
```

</details>

---
## Part 2: Event Selection and Cuts (45 min)

### Beginner Level
1. Apply kinematic cuts: pT > 20 GeV, |η| < 2.4
2. Count events before/after cuts
3. Visualize the effect of cuts

In [ ]:
# TODO: Apply kinematic cuts

# Define cuts
PT_MIN = 20.0  # GeV
ETA_MAX = 2.4

# YOUR CODE HERE
# Create boolean masks for each cut
# mask_pt = ...
# mask_eta = ...
# mask_all = mask_pt & mask_eta

# Apply cuts
# data_selected = data[mask_all].copy()

# Print cut-flow
# print(f"Events before cuts: {len(data)}")
# print(f"Events after pT cut: {mask_pt.sum()}")
# print(f"Events after eta cut: {mask_eta.sum()}")
# print(f"Events after all cuts: {len(data_selected)}")

<details>
<summary>💡 Click to reveal solution</summary>

```python
# Define cuts
PT_MIN = 20.0  # GeV
ETA_MAX = 2.4

# Create boolean masks
mask_pt = (data['mu1_pt'] > PT_MIN) & (data['mu2_pt'] > PT_MIN)
mask_eta = (np.abs(data['mu1_eta']) < ETA_MAX) & (np.abs(data['mu2_eta']) < ETA_MAX)
mask_all = mask_pt & mask_eta

# Apply cuts
data_selected = data[mask_all].copy()

# Print cut-flow
print("=== Cut-Flow Table ===")
print(f"Initial events:     {len(data):6d} (100.0%)")
print(f"After pT > 20 GeV:  {mask_pt.sum():6d} ({100*mask_pt.sum()/len(data):.1f}%)")
print(f"After |η| < 2.4:    {mask_eta.sum():6d} ({100*mask_eta.sum()/len(data):.1f}%)")
print(f"After all cuts:     {len(data_selected):6d} ({100*len(data_selected)/len(data):.1f}%)")

# Apply same cuts to MC
mc_mask = ((mc_data['mu1_pt'] > PT_MIN) & (mc_data['mu2_pt'] > PT_MIN) &
           (np.abs(mc_data['mu1_eta']) < ETA_MAX) & (np.abs(mc_data['mu2_eta']) < ETA_MAX))
mc_selected = mc_data[mc_mask].copy()
print(f"\nMC events after cuts: {len(mc_selected)}")
```

</details>

In [ ]:
# TODO: Visualize the effect of cuts on mass distribution

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# YOUR CODE HERE
# Left: Before cuts
# Right: After cuts

plt.tight_layout()
plt.show()

<details>
<summary>💡 Click to reveal solution</summary>

```python
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

mass_range = (60, 120)
bins = 40

# Before cuts
axes[0].hist(data['mass'], bins=bins, range=mass_range, histtype='step',
             linewidth=2, color='blue', label='All events')
axes[0].axvline(91.2, color='red', linestyle='--', alpha=0.7)
axes[0].set_xlabel(r'$m_{\mu\mu}$ (GeV/c²)', fontsize=12)
axes[0].set_ylabel('Events', fontsize=12)
axes[0].set_title('Before Cuts', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# After cuts
axes[1].hist(data_selected['mass'], bins=bins, range=mass_range, histtype='step',
             linewidth=2, color='green', label='After cuts')
axes[1].axvline(91.2, color='red', linestyle='--', alpha=0.7)
axes[1].set_xlabel(r'$m_{\mu\mu}$ (GeV/c²)', fontsize=12)
axes[1].set_ylabel('Events', fontsize=12)
axes[1].set_title('After Cuts', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# The Z peak should be more prominent after cuts!
```

</details>

---
## Part 3: Functions and Classes (45 min)

### Beginner Level
Create reusable functions and a simple class for the analysis.

In [ ]:
# TODO: Create a function library

def calculate_invariant_mass_vectorized(pt1, eta1, phi1, pt2, eta2, phi2):
    """
    Calculate invariant mass (vectorized version for arrays).
    
    Parameters:
    -----------
    pt1, eta1, phi1 : array-like
        Kinematics of first muon
    pt2, eta2, phi2 : array-like
        Kinematics of second muon
    
    Returns:
    --------
    np.ndarray : Invariant masses
    """
    # YOUR CODE HERE
    pass

def apply_cuts(df, pt_min=20.0, eta_max=2.4):
    """
    Apply kinematic cuts to DataFrame.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input data
    pt_min : float
        Minimum pT cut (GeV)
    eta_max : float
        Maximum |η| cut
    
    Returns:
    --------
    pd.DataFrame : Filtered data
    """
    # YOUR CODE HERE
    pass

def plot_mass_spectrum(masses, bins=50, range=(60, 120), ax=None, **kwargs):
    """
    Plot invariant mass spectrum with error bars.
    
    Parameters:
    -----------
    masses : array-like
        Invariant mass values
    bins : int
        Number of histogram bins
    range : tuple
        (min, max) for histogram range
    ax : matplotlib.axes.Axes, optional
        Axes to plot on
    **kwargs : dict
        Additional arguments for histogram
    
    Returns:
    --------
    tuple : (counts, bin_edges, ax)
    """
    # YOUR CODE HERE
    pass

# Test your functions
# masses = calculate_invariant_mass_vectorized(
#     data['mu1_pt'], data['mu1_eta'], data['mu1_phi'],
#     data['mu2_pt'], data['mu2_eta'], data['mu2_phi']
# )
# print(f"Calculated {len(masses)} masses")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def calculate_invariant_mass_vectorized(pt1, eta1, phi1, pt2, eta2, phi2):
    """
    Calculate invariant mass (vectorized version for arrays).
    """
    pt1 = np.asarray(pt1)
    pt2 = np.asarray(pt2)
    eta1 = np.asarray(eta1)
    eta2 = np.asarray(eta2)
    phi1 = np.asarray(phi1)
    phi2 = np.asarray(phi2)
    
    deta = eta1 - eta2
    dphi = phi1 - phi2
    
    m2 = 2 * pt1 * pt2 * (np.cosh(deta) - np.cos(dphi))
    return np.sqrt(np.maximum(m2, 0))

def apply_cuts(df, pt_min=20.0, eta_max=2.4):
    """
    Apply kinematic cuts to DataFrame.
    """
    mask = (
        (df['mu1_pt'] > pt_min) &
        (df['mu2_pt'] > pt_min) &
        (np.abs(df['mu1_eta']) < eta_max) &
        (np.abs(df['mu2_eta']) < eta_max)
    )
    return df[mask].copy()

def plot_mass_spectrum(masses, bins=50, range=(60, 120), ax=None, **kwargs):
    """
    Plot invariant mass spectrum with error bars.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    
    counts, bin_edges, _ = ax.hist(masses, bins=bins, range=range,
                                    histtype='step', linewidth=2, **kwargs)
    
    # Add error bars
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    errors = np.sqrt(counts)
    ax.errorbar(bin_centers, counts, yerr=errors, fmt='none',
                capsize=2, color=kwargs.get('color', 'black'))
    
    ax.set_xlabel(r'$m_{\mu\mu}$ (GeV/c²)', fontsize=12)
    ax.set_ylabel('Events', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    return counts, bin_edges, ax

# Test
masses = calculate_invariant_mass_vectorized(
    data['mu1_pt'], data['mu1_eta'], data['mu1_phi'],
    data['mu2_pt'], data['mu2_eta'], data['mu2_phi']
)
print(f"Calculated {len(masses)} masses, range: [{masses.min():.1f}, {masses.max():.1f}] GeV")
```

</details>

In [ ]:
# TODO: Create a MuonPair class

class MuonPair:
    """
    A pair of muons from a collision event.
    """
    
    def __init__(self, pt1, eta1, phi1, pt2, eta2, phi2):
        """
        Initialize muon pair with kinematics.
        """
        # YOUR CODE HERE
        pass
    
    def invariant_mass(self):
        """
        Calculate invariant mass of the pair.
        """
        # YOUR CODE HERE
        pass
    
    def passes_cuts(self, pt_min=20.0, eta_max=2.4):
        """
        Check if pair passes kinematic cuts.
        """
        # YOUR CODE HERE
        pass
    
    def __repr__(self):
        mass = self.invariant_mass()
        return f"MuonPair(mass={mass:.2f} GeV)"

# Test
# pair = MuonPair(pt1=45, eta1=0.5, phi1=0.1, pt2=40, eta2=-0.3, phi2=3.0)
# print(pair)
# print(f"Mass: {pair.invariant_mass():.2f} GeV")
# print(f"Passes cuts: {pair.passes_cuts()}")

<details>
<summary>💡 Click to reveal solution</summary>

```python
class MuonPair:
    """
    A pair of muons from a collision event.
    """
    
    def __init__(self, pt1, eta1, phi1, pt2, eta2, phi2):
        """
        Initialize muon pair with kinematics.
        """
        self.mu1 = {'pt': pt1, 'eta': eta1, 'phi': phi1}
        self.mu2 = {'pt': pt2, 'eta': eta2, 'phi': phi2}
    
    def invariant_mass(self):
        """
        Calculate invariant mass of the pair.
        """
        deta = self.mu1['eta'] - self.mu2['eta']
        dphi = self.mu1['phi'] - self.mu2['phi']
        m2 = 2 * self.mu1['pt'] * self.mu2['pt'] * (np.cosh(deta) - np.cos(dphi))
        return np.sqrt(m2) if m2 > 0 else 0
    
    def passes_cuts(self, pt_min=20.0, eta_max=2.4):
        """
        Check if pair passes kinematic cuts.
        """
        pt_ok = (self.mu1['pt'] > pt_min) and (self.mu2['pt'] > pt_min)
        eta_ok = (abs(self.mu1['eta']) < eta_max) and (abs(self.mu2['eta']) < eta_max)
        return pt_ok and eta_ok
    
    def __repr__(self):
        mass = self.invariant_mass()
        return f"MuonPair(mass={mass:.2f} GeV)"

# Test
pair = MuonPair(pt1=45, eta1=0.5, phi1=0.1, pt2=40, eta2=-0.3, phi2=3.0)
print(pair)
print(f"Mass: {pair.invariant_mass():.2f} GeV")
print(f"Passes cuts: {pair.passes_cuts()}")
```

</details>

---
## Part 4: Error Handling (30 min)

Add robust error handling to your analysis functions.

In [ ]:
# TODO: Add error handling to your functions

def calculate_invariant_mass_safe(pt1, eta1, phi1, pt2, eta2, phi2):
    """
    Calculate invariant mass with validation.
    
    Raises:
    -------
    ValueError : if inputs are invalid (negative pT, unphysical eta)
    """
    # YOUR CODE HERE
    # 1. Convert to arrays
    # 2. Validate pT >= 0
    # 3. Validate |eta| is reasonable (< 10)
    # 4. Calculate mass
    # 5. Return mass (with NaN for any invalid entries)
    
    pass

# Test with valid data
# mass = calculate_invariant_mass_safe(30, 0.5, 0.1, 25, -0.3, 2.5)
# print(f"Valid input: mass = {mass:.2f} GeV")

# Test with invalid data
# try:
#     mass = calculate_invariant_mass_safe(-10, 0.5, 0.1, 25, -0.3, 2.5)
# except ValueError as e:
#     print(f"Caught error: {e}")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def calculate_invariant_mass_safe(pt1, eta1, phi1, pt2, eta2, phi2):
    """
    Calculate invariant mass with validation.
    """
    # Convert to arrays
    pt1 = np.asarray(pt1, dtype=float)
    pt2 = np.asarray(pt2, dtype=float)
    eta1 = np.asarray(eta1, dtype=float)
    eta2 = np.asarray(eta2, dtype=float)
    phi1 = np.asarray(phi1, dtype=float)
    phi2 = np.asarray(phi2, dtype=float)
    
    # Validate pT
    if np.any(pt1 < 0) or np.any(pt2 < 0):
        raise ValueError("pT must be non-negative")
    
    # Validate eta
    if np.any(np.abs(eta1) > 10) or np.any(np.abs(eta2) > 10):
        raise ValueError("Unphysical eta values (|eta| > 10)")
    
    # Check for NaN
    all_values = [pt1, pt2, eta1, eta2, phi1, phi2]
    if any(np.any(np.isnan(v)) for v in all_values):
        print("Warning: NaN values in input")
    
    # Calculate mass
    deta = eta1 - eta2
    dphi = phi1 - phi2
    m2 = 2 * pt1 * pt2 * (np.cosh(deta) - np.cos(dphi))
    
    return np.sqrt(np.maximum(m2, 0))

# Test with valid data
mass = calculate_invariant_mass_safe(30, 0.5, 0.1, 25, -0.3, 2.5)
print(f"Valid input: mass = {mass:.2f} GeV")

# Test with invalid data
try:
    mass = calculate_invariant_mass_safe(-10, 0.5, 0.1, 25, -0.3, 2.5)
except ValueError as e:
    print(f"Caught error: {e}")
```

</details>

---
## Part 5: Testing (30 min)

Write tests for your analysis code.

In [ ]:
# TODO: Write test functions

def test_invariant_mass_z_boson():
    """Test that back-to-back muons give Z mass."""
    # YOUR CODE HERE
    pass

def test_invariant_mass_positive():
    """Test that mass is always non-negative."""
    # YOUR CODE HERE
    pass

def test_apply_cuts_reduces_events():
    """Test that cuts reduce number of events."""
    # YOUR CODE HERE
    pass

def test_muon_pair_class():
    """Test MuonPair class methods."""
    # YOUR CODE HERE
    pass

# Run tests
print("Running tests...\n")
# test_invariant_mass_z_boson()
# test_invariant_mass_positive()
# test_apply_cuts_reduces_events()
# test_muon_pair_class()
print("\n✓ All tests passed!")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def test_invariant_mass_z_boson():
    """Test that back-to-back muons give Z mass."""
    mass = calculate_invariant_mass_vectorized(
        pt1=45.6, eta1=0, phi1=0,
        pt2=45.6, eta2=0, phi2=np.pi
    )
    assert abs(mass - 91.2) < 1.0, f"Expected ~91.2 GeV, got {mass:.2f}"
    print("✓ test_invariant_mass_z_boson passed")

def test_invariant_mass_positive():
    """Test that mass is always non-negative."""
    test_cases = [
        (30, 0.5, 0.1, 25, -0.3, 2.5),
        (100, 2.0, -1.5, 50, -1.0, 1.0),
        (10, 0, 0, 10, 0, 0),  # Collinear
    ]
    
    for pt1, eta1, phi1, pt2, eta2, phi2 in test_cases:
        mass = calculate_invariant_mass_vectorized(pt1, eta1, phi1, pt2, eta2, phi2)
        assert mass >= 0, f"Mass should be non-negative, got {mass}"
    
    print("✓ test_invariant_mass_positive passed")

def test_apply_cuts_reduces_events():
    """Test that cuts reduce number of events."""
    # Create test data with some events that should fail cuts
    test_df = pd.DataFrame({
        'mu1_pt': [10, 30, 50],  # First fails pT cut
        'mu2_pt': [40, 40, 40],
        'mu1_eta': [0.5, 3.0, 0.5],  # Second fails eta cut
        'mu2_eta': [0.3, 0.3, 0.3],
    })
    
    selected = apply_cuts(test_df, pt_min=20, eta_max=2.4)
    
    assert len(selected) < len(test_df), "Cuts should reduce events"
    assert len(selected) == 1, f"Expected 1 event, got {len(selected)}"
    
    print("✓ test_apply_cuts_reduces_events passed")

def test_muon_pair_class():
    """Test MuonPair class methods."""
    pair = MuonPair(pt1=45, eta1=0.5, phi1=0.1, pt2=40, eta2=-0.3, phi2=3.0)
    
    # Test mass calculation
    mass = pair.invariant_mass()
    assert mass > 0, "Mass should be positive"
    assert np.isfinite(mass), "Mass should be finite"
    
    # Test passes_cuts
    assert pair.passes_cuts(pt_min=20, eta_max=2.4) == True
    assert pair.passes_cuts(pt_min=50, eta_max=2.4) == False  # Fails pT
    
    print("✓ test_muon_pair_class passed")

# Run tests
print("Running tests...\n")
test_invariant_mass_z_boson()
test_invariant_mass_positive()
test_apply_cuts_reduces_events()
test_muon_pair_class()
print("\n✓ All tests passed!")
```

</details>

---
## Part 6: Final Analysis and Visualization (30 min)

Bring everything together for the final analysis.

In [ ]:
# TODO: Create final publication-quality plot

fig, axes = plt.subplots(2, 1, figsize=(10, 10),
                         gridspec_kw={'height_ratios': [3, 1]},
                         sharex=True)

# YOUR CODE HERE
# Top panel: Data histogram with error bars + MC comparison
# Bottom panel: Data/MC ratio

# Add labels, legend, title
# Save figure

plt.tight_layout()
plt.show()

<details>
<summary>💡 Click to reveal solution</summary>

```python
# Apply cuts to both data and MC
data_final = apply_cuts(data)
mc_final = apply_cuts(mc_data)

fig, axes = plt.subplots(2, 1, figsize=(10, 10),
                         gridspec_kw={'height_ratios': [3, 1]},
                         sharex=True)

# Define binning
bins = np.linspace(60, 120, 41)
bin_centers = (bins[:-1] + bins[1:]) / 2
bin_width = bins[1] - bins[0]

# ===== Top panel =====
ax1 = axes[0]

# Data histogram
data_counts, _ = np.histogram(data_final['mass'], bins=bins)
data_errors = np.sqrt(data_counts)
ax1.errorbar(bin_centers, data_counts, yerr=data_errors,
             fmt='ko', markersize=4, label='Data')

# MC histogram (scaled to data)
mc_counts, _ = np.histogram(mc_final['mass'], bins=bins)
scale = data_counts.sum() / mc_counts.sum()
mc_scaled = mc_counts * scale

ax1.bar(bin_centers, mc_scaled, width=bin_width, alpha=0.5,
        color='blue', label='Monte Carlo')

# Formatting
ax1.set_ylabel('Events / 1.5 GeV', fontsize=14)
ax1.set_title(r'$Z \rightarrow \mu^+\mu^-$: Data vs Monte Carlo', fontsize=16)
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# Add experiment label
ax1.text(0.05, 0.95, 'Course Project\n' + r'$\sqrt{s}$ = 13 TeV',
         transform=ax1.transAxes, verticalalignment='top',
         fontsize=12, family='sans-serif')

# ===== Bottom panel (ratio) =====
ax2 = axes[1]

ratio = np.divide(data_counts, mc_scaled, where=mc_scaled > 0)
ratio_err = np.divide(data_errors, mc_scaled, where=mc_scaled > 0)

ax2.errorbar(bin_centers, ratio, yerr=ratio_err, fmt='ko', markersize=4)
ax2.axhline(1.0, color='red', linestyle='--', linewidth=1)
ax2.fill_between([60, 120], [0.9, 0.9], [1.1, 1.1], alpha=0.2, color='gray')

ax2.set_xlabel(r'$m_{\mu\mu}$ (GeV/c²)', fontsize=14)
ax2.set_ylabel('Data / MC', fontsize=14)
ax2.set_ylim(0.5, 1.5)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('z_peak_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved as 'z_peak_analysis.png'")
```

</details>

In [ ]:
# TODO: Extract Z mass peak properties

from scipy.optimize import curve_fit
from scipy.stats import norm

def gaussian_plus_background(x, A, mu, sigma, a, b):
    """Gaussian signal + linear background."""
    return A * norm.pdf(x, mu, sigma) + a * x + b

# YOUR CODE HERE
# 1. Create histogram of selected data in Z mass region
# 2. Fit with Gaussian + background
# 3. Extract peak position and width
# 4. Calculate significance

# Print results
# print("=== Fit Results ===")
# print(f"Z mass: {mu:.2f} ± {mu_err:.2f} GeV")
# print(f"Width:  {sigma:.2f} ± {sigma_err:.2f} GeV")

<details>
<summary>💡 Click to reveal solution</summary>

```python
from scipy.optimize import curve_fit
from scipy.stats import norm

def gaussian_plus_background(x, A, mu, sigma, a, b):
    """Gaussian signal + linear background."""
    return A * norm.pdf(x, mu, sigma) + a * x + b

# Create histogram
mass_data = data_final['mass'].values
mass_range = (70, 110)
bins = 40

counts, bin_edges = np.histogram(mass_data, bins=bins, range=mass_range)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
errors = np.sqrt(counts)
errors[errors == 0] = 1  # Avoid division by zero

# Initial guess
p0 = [counts.max() * 3, 91.2, 2.5, 0, counts.mean()]

# Fit
try:
    popt, pcov = curve_fit(gaussian_plus_background, bin_centers, counts,
                           p0=p0, sigma=errors, absolute_sigma=True)
    perr = np.sqrt(np.diag(pcov))
    
    A, mu, sigma, a, b = popt
    A_err, mu_err, sigma_err, a_err, b_err = perr
    
    # Print results
    print("=== Fit Results ===")
    print(f"Z mass: {mu:.2f} ± {mu_err:.2f} GeV")
    print(f"Width:  {sigma:.2f} ± {sigma_err:.2f} GeV")
    print(f"Amplitude: {A:.1f} ± {A_err:.1f}")
    
    # Calculate significance
    # Signal in ±2σ window
    mask_signal = (mass_data > mu - 2*sigma) & (mass_data < mu + 2*sigma)
    n_signal_region = mask_signal.sum()
    n_background = b * 4 * sigma  # Linear background estimate
    significance = (n_signal_region - n_background) / np.sqrt(n_signal_region)
    print(f"\nSignificance: {significance:.1f} σ")
    
    # Plot fit
    fig, ax = plt.subplots(figsize=(10, 7))
    
    ax.errorbar(bin_centers, counts, yerr=errors, fmt='ko', label='Data')
    
    x_fit = np.linspace(mass_range[0], mass_range[1], 200)
    ax.plot(x_fit, gaussian_plus_background(x_fit, *popt), 'r-', 
            linewidth=2, label='Fit: Gaussian + background')
    ax.plot(x_fit, a*x_fit + b, 'b--', linewidth=1, label='Background')
    
    ax.set_xlabel(r'$m_{\mu\mu}$ (GeV/c²)', fontsize=14)
    ax.set_ylabel('Events', fontsize=14)
    ax.set_title('Z Peak Fit', fontsize=16)
    
    # Add fit results text
    text = (f"$m_Z = {mu:.2f} \\pm {mu_err:.2f}$ GeV\n"
            f"$\\sigma = {sigma:.2f} \\pm {sigma_err:.2f}$ GeV")
    ax.text(0.95, 0.95, text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except RuntimeError as e:
    print(f"Fit failed: {e}")
```

</details>

---
## Summary and Conclusions

Write a brief summary of your analysis:

In [ ]:
# Summary
print("="*50)
print("     ANALYSIS SUMMARY")
print("="*50)
print(f"\nDataset: {len(data)} dimuon events")
print(f"After cuts: {len(data_final)} events ({100*len(data_final)/len(data):.1f}%)")
print(f"\nKinematic cuts applied:")
print(f"  - pT > {PT_MIN} GeV")
print(f"  - |η| < {ETA_MAX}")
# print(f"\nZ boson mass: {mu:.2f} ± {mu_err:.2f} GeV")
# print(f"Z boson width: {sigma:.2f} ± {sigma_err:.2f} GeV")
# print(f"\nSignificance: {significance:.1f} σ")
print("\n" + "="*50)

---
## Congratulations!

You have completed the final project! You have:

✅ Loaded and explored particle physics data  
✅ Applied kinematic selection cuts  
✅ Created reusable functions and classes  
✅ Added error handling and validation  
✅ Written tests for your code  
✅ Created publication-quality visualizations  
✅ Extracted physics results (Z boson mass)

**Well done!** 🎉

---

### Next Steps

To continue learning:
- Explore the [Scikit-HEP ecosystem](https://scikit-hep.org/)
- Learn [uproot](https://uproot.readthedocs.io/) for ROOT file I/O
- Practice with real data from [CERN Open Data](https://opendata.cern.ch/)
- Join the [HEP Software Foundation](https://hepsoftwarefoundation.org/) community